# Multiple Linear Regression (Manual Calculation VS Scikit-Learn Lib)


In this notebook, we will learn **Simple Linear Regression** in two ways:  

1. **From Scratch** – implementing formulas step by step.  
2. **With Scikit-learn** – using Python’s machine learning library.  

## 1. Dataset

We use the same housing example with two predictors (area and bedrooms) and one response (price):

| House | Area (x₁) | Bedrooms (x₂) | Price (y) |
|---:|---:|---:|---:|
| 1 | 1000 | 2 | 250 |
| 2 | 1500 | 3 | 400 |
| 3 | 2000 | 4 | 450 |
| 4 | 2500 | 3 | 500 |
| 5 | 3000 | 5 | 550 |

In [4]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

data = {
    'x1_area': [1000, 1500, 2000, 2500, 3000],
    'x2_bedrooms': [2, 3, 4, 3, 5],
    'y_price': [250, 400, 450, 500, 550]
}
df = pd.DataFrame(data)
df

,x1_area,x2_bedrooms,y_price
0,1000,2,250
1,1500,3,400
2,2000,4,450
3,2500,3,500
4,3000,5,550


## 2. Simple Linear Regression from Scratch

## Step 1 — Compute sample means

Calculate sample means:
$$\bar{x}_1 = \frac{1}{n}\sum_{i=1}^n x_{1i},\qquad \bar{x}_2 = \frac{1}{n}\sum_{i=1}^n x_{2i},\qquad \bar{y} = \frac{1}{n}\sum_{i=1}^n y_i$$

In [7]:
x1 = df['x1_area']
x2 = df['x2_bedrooms']
y = df['y_price']

x1_bar = x1.mean()
x2_bar = x2.mean()
y_bar = y.mean()

x1_bar, x2_bar, y_bar

(2000.0, 3.4, 430.0)

## Step 2 — Deviations (show table)

Compute deviations:
$$x_{1i}' = x_{1i} - \bar{x}_1,\qquad x_{2i}' = x_{2i} - \bar{x}_2,\qquad y_i' = y_i - \bar{y}$$

In [9]:
df['x1_prime'] = x1 - x1_bar
df['x2_prime'] = x2 - x2_bar
df['y_prime'] = y - y_bar

df[['x1_area','x2_bedrooms','y_price','x1_prime','x2_prime','y_prime']]

,x1_area,x2_bedrooms,y_price,x1_prime,x2_prime,y_prime
0,1000,2,250,-1000.0,-1.4,-180.0
1,1500,3,400,-500.0,-0.4,-30.0
2,2000,4,450,0.0,0.6,20.0
3,2500,3,500,500.0,-0.4,70.0
4,3000,5,550,1000.0,1.6,120.0


## Step 3 — Cross-products (row-wise) and totals

Compute the row-wise columns:
$$(x'_{1i})^2,\quad (x'_{2i})^2,\quad x'_{1i}x'_{2i},\quad x'_{1i}y'_i,\quad x'_{2i}y'_i$$

Then compute the sums (needed for normal equations):
$$S_{x_1x_1}=\sum (x'_{1i})^2,\ S_{x_2x_2}=\sum (x'_{2i})^2,\ S_{x_1x_2}=\sum x'_{1i}x'_{2i},\ S_{x_1y}=\sum x'_{1i}y'_i,\ S_{x_2y}=\sum x'_{2i}y'_i$$

In [11]:
df['x1p_sq'] = df['x1_prime']**2
df['x2p_sq'] = df['x2_prime']**2
df['x1px2p'] = df['x1_prime']*df['x2_prime']
df['x1py'] = df['x1_prime']*df['y_prime']
df['x2py'] = df['x2_prime']*df['y_prime']

cols = ['x1_prime','x2_prime','y_prime','x1p_sq','x2p_sq','x1px2p','x1py','x2py']
df[cols]

,x1_prime,x2_prime,y_prime,x1p_sq,x2p_sq,x1px2p,x1py,x2py
0,-1000.0,-1.4,-180.0,1000000.0,1.96,1400.0,180000.0,252.0
1,-500.0,-0.4,-30.0,250000.0,0.16,200.0,15000.0,12.0
2,0.0,0.6,20.0,0.0,0.36,0.0,0.0,12.0
3,500.0,-0.4,70.0,250000.0,0.16,-200.0,35000.0,-28.0
4,1000.0,1.6,120.0,1000000.0,2.56,1600.0,120000.0,192.0


In [12]:
S_x1x1 = df['x1p_sq'].sum()
S_x2x2 = df['x2p_sq'].sum()
S_x1x2 = df['x1px2p'].sum()
S_x1y = df['x1py'].sum()
S_x2y = df['x2py'].sum()

pd.DataFrame({
    'S_x1x1':[S_x1x1],
    'S_x2x2':[S_x2x2],
    'S_x1x2':[S_x1x2],
    'S_x1y':[S_x1y],
    'S_x2y':[S_x2y]
})

,S_x1x1,S_x2x2,S_x1x2,S_x1y,S_x2y
0,2500000.0,5.2,3000.0,350000.0,440.0


## Step 4 — Normal equations (centered form)

Write the centered normal equations (for slopes):
$$S_{x_1x_1}b_1 + S_{x_1x_2}b_2 = S_{x_1y}$$
$$S_{x_1x_2}b_1 + S_{x_2x_2}b_2 = S_{x_2y}$$

In [19]:
A = np.array([[S_x1x1, S_x1x2],[S_x1x2, S_x2x2]])
b_vec = np.array([S_x1y, S_x2y])
b1, b2 = np.linalg.solve(A, b_vec)
b1, b2

(0.12500000000000003, 12.499999999999982)

## Step 5 — Intercept

Compute intercept using means:
$$b_0 = \bar{y} - b_1\bar{x}_1 - b_2\bar{x}_2$$

In [22]:
b0 = y_bar - b1*x1_bar - b2*x2_bar
b0

137.5

## Final regression equation
$$\hat{y} = b_0 + b_1 x_1 + b_2 x_2$$

In [25]:
print(f"Manual model: y = {b0:.6f} + {b1:.6f}*x1 + {b2:.6f}*x2")

Manual model: y = 137.500000 + 0.125000*x1 + 12.500000*x2


## 3. Prediction example
Predict for a house with $x_1=2200$ and $x_2=4$.

In [28]:
def predict_manual(x1_val, x2_val):
    return b0 + b1*x1_val + b2*x2_val

predict_manual(2200, 4)

462.5

## 4. Validation — matrix formula (NumPy)
$$\beta = (X^TX)^{-1}X^Ty$$

In [31]:
X = np.column_stack((np.ones(len(x1)), x1, x2))
theta_hat = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
theta_hat  # [b0, b1, b2]

array([1.375e+02, 1.250e-01, 1.250e+01])

## Validation — scikit-learn
Fit `LinearRegression` and show intercept & coefficients.

In [34]:
model = LinearRegression()
model.fit(df[['x1_area','x2_bedrooms']], df['y_price'])
print('sklearn intercept =', model.intercept_)
print('sklearn coef (b1,b2) =', model.coef_)

sklearn intercept = 137.49999999999983
sklearn coef (b1,b2) = [ 0.125 12.5  ]


## 5. Error metrics — Manual model
Compute RSS, MSE, RMSE, R² for manual predictions.

In [37]:
y_pred_manual = predict_manual(x1, x2)
resid_manual = y - y_pred_manual
RSS_manual = np.sum(resid_manual**2)
MSE_manual = mean_squared_error(y, y_pred_manual)
RMSE_manual = np.sqrt(MSE_manual)
R2_manual = r2_score(y, y_pred_manual)

print('Manual model metrics:')
print('RSS =', RSS_manual)
print('MSE =', MSE_manual)
print('RMSE =', RMSE_manual)
print('R2 =', R2_manual)

Manual model metrics:
RSS = 3749.9999999999945
MSE = 749.9999999999989
RMSE = 27.386127875258286
R2 = 0.929245283018868


## Error metrics — scikit-learn model
Compute metrics for sklearn predictions.

In [40]:
y_pred_sklearn = model.predict(df[['x1_area','x2_bedrooms']])
resid_sklearn = y - y_pred_sklearn
RSS_sklearn = np.sum(resid_sklearn**2)
MSE_sklearn = mean_squared_error(y, y_pred_sklearn)
RMSE_sklearn = np.sqrt(MSE_sklearn)
R2_sklearn = r2_score(y, y_pred_sklearn)

print('Sklearn model metrics:')
print('RSS =', RSS_sklearn)
print('MSE =', MSE_sklearn)
print('RMSE =', RMSE_sklearn)
print('R2 =', R2_sklearn)

Sklearn model metrics:
RSS = 3749.9999999999945
MSE = 749.9999999999989
RMSE = 27.386127875258286
R2 = 0.929245283018868


## 6. Final comparison (Manual vs sklearn)
Pandas DataFrame comparing coefficients and metrics.

In [43]:
manual_row = {
    'b0': b0,
    'b1': b1,
    'b2': b2,
    'RSS': RSS_manual,
    'MSE': MSE_manual,
    'RMSE': RMSE_manual,
    'R2': R2_manual
}
sk_row = {
    'b0': model.intercept_,
    'b1': model.coef_[0],
    'b2': model.coef_[1],
    'RSS': RSS_sklearn,
    'MSE': MSE_sklearn,
    'RMSE': RMSE_sklearn,
    'R2': R2_sklearn
}
compare_df = pd.DataFrame([manual_row, sk_row], index=['manual','sklearn'])
compare_df.round(6)

,b0,b1,b2,RSS,MSE,RMSE,R2
manual,137.5,0.125,12.5,3750.0,750.0,27.386128,0.929245
sklearn,137.5,0.125,12.5,3750.0,750.0,27.386128,0.929245
